<a href="https://colab.research.google.com/github/nurfnick/Metaheuristics/blob/main/Homework7/Group20GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
#basic genetic algorithm Python code provided as base code for the DSA/ISE 5113 course
#author: Charles Nicholson
#date: 4/5/2019

#NOTE: You will need to change various parts of this code.  However, please keep the majority of the code intact (e.g., you may revise existing logic/functions and add new logic/functions, but don't completely rewrite the entire base code!)
#However, I would like all students to have the same problem instance, therefore please do not change anything relating to:
#   random number generation
#   number of items (should be 150)
#   random problem instance
#   weight limit of the knapsack

#------------------------------------------------------------------------------

#Student name:Nicholas Jacob
#Date: 4/12/24


#need some python libraries
import copy
import math
from random import Random
import numpy as np



#to setup a random number generator, we will specify a "seed" value
#need this for the random number generation -- do not change
seed = 51132023
myPRNG = Random(seed)

#to get a random number between 0 and 1, use this:             myPRNG.random()
#to get a random number between lwrBnd and upprBnd, use this:  myPRNG.uniform(lwrBnd,upprBnd)
#to get a random integer between lwrBnd and upprBnd, use this: myPRNG.randint(lwrBnd,upprBnd)

#number of elements in a solution
n = 150

#create an "instance" for the knapsack problem
value = []
for i in range(0,n):
    #value.append(round(myPRNG.expovariate(1/500)+1,1))
    value.append(round(myPRNG.triangular(150,2000,500),1))

weights = []
for i in range(0,n):
    weights.append(round(myPRNG.triangular(8,300,95),1))

#define max weight for the knapsack
maxWeight = 2500


#change anything you like below this line, but keep the gist of the program ------------------------------------

#monitor the number of solutions evaluated
solutionsChecked = 0


populationSize = 200 #size of GA population
Generations = 100   #number of GA generations

crossOverRate = 0.95
mutationRate = 0.5
eliteSolutions = 25



#create an continuous valued chromosome
def createChromosome(n):
    #this code as-is expects chromosomes to be stored as a list, e.g., x = []
    #write code to generate chromosomes, most likely want this to be randomly generated

    x = []   #i recommend creating the solution as a list
    for i in range(n):
      x.append(myPRNG.randint(0,1)) #pick a random 1 or 0 until you fill the list

    #i = myPRNG.randint(0,n-1)
   # while evaluate(x)[1] > maxWeight:
    #  x[i] = 0
    #  i=myPRNG.randint(0,n-1)

    return x


#create initial population by calling the "createChromosome" function many times and adding each to a list of chromosomes (a.k.a., the "population")
def initializePopulation(): #n is size of population; d is dimensions of chromosome

    population = []
    populationFitness = []

    for i in range(populationSize):
        population.append(createChromosome(n))
        populationFitness.append(evaluate(population[i]))

    tempZip = zip(population, populationFitness)
    popVals = sorted(tempZip, key=lambda tempZip: tempZip[1], reverse = True)

    #the return object is a reversed sorted list of tuples:
    #the first element of the tuple is the chromosome; the second element is the fitness value
    #for example:  popVals[0] is represents the best individual in the population
    #popVals[0] for a 2D problem might be  ([-70.2, 426.1], 483.3)  -- chromosome is the list [-70.2, 426.1] and the fitness is 483.3

    return popVals

#implement a crossover
def crossover(x1,x2):
  p = myPRNG.random()
  if p>crossOverRate: #just go back into the gene pool as is
    offspring1 = x1[:]
    offspring2 = x2[:]
  else: #have some children
    cutIndex = myPRNG.randint(1,n-2) #cut at a place where you indeed make new arrays.
    offspring1 = x1[0:cutIndex] + x2[cutIndex:]
    offspring2 = x2[0:cutIndex] + x1[cutIndex:]

        #with some probability (i.e., crossoverRate) perform breeding via crossover,
        #i.e. two parents (x1 and x2) should produce two offsrping (offspring1 and offspring2)
        # --- the first part of offspring1 comes from x1, and the second part of offspring1 comes from x2
        # --- the first part of offspring2 comes from x2, and the second part of offspring2 comes from x1

        #if no breeding occurs, then offspring1 and offspring2 can simply be copies of x1 and x2, respectively



  return offspring1, offspring2  #two offspring are returned


#function to compute the weight of chromosome x
def calcWeight(x):

    a=np.array(x)
    c=np.array(weights)

    totalWeight = np.dot(a,c)    #compute the weight value of the knapsack selection

    return totalWeight   #returns total weight


#function to determine how many items have been selected in a particular chromosome x
def itemsSelected(x):

    a=np.array(x)
    return np.sum(a)   #returns total number of items selected



#function to evaluate a solution x
def evaluate(x):

    a=np.array(x)
    b=np.array(value)

    totalValue = np.dot(a,b)     #compute the value of the knapsack selection
    totalWeight = calcWeight(x) #compute the total weight

    #you will VERY LIKELY need to add some penalties or sometype of modification of the totalvalue to compute the chromosome fitness
    #for instance, you may include penalties if the knapsack weight exceeds the maximum allowed weight

    fitness  = totalValue

    if totalWeight > maxWeight:
      fitness = fitness - 100*(totalWeight - maxWeight)#penalty of 100 per pound over weight

    return fitness   #returns the chromosome fitness



In [110]:

#performs tournament selection; k chromosomes are selected (with repeats allowed) and the best advances to the mating pool
#function returns the mating pool with size equal to the initial population
def tournamentSelection(pop,k):

    #randomly select k chromosomes; the best joins the mating pool
    matingPool = []

    while len(matingPool)<populationSize:

        ids = [myPRNG.randint(0,populationSize-1) for i in range(k)]
        competingIndividuals = [pop[i][1] for i in ids]
        bestID=ids[competingIndividuals.index(max(competingIndividuals))]
        matingPool.append(pop[bestID][0])

    return matingPool


def rouletteWheel(pop):

    matingPool = [] #initialize an empty array
    rank = [i/(len(pop)*(len(pop)+1)/2) for i in range(len(pop),0,-1)] #get a probability based on rank
    cumrank = np.cumsum(rank) #needed cummulative probability of those ranks.

    while len(matingPool)<populationSize: #fill up the whole mating pool
      p = myPRNG.random() #get a rando between 0 and 1
      i = 0 #start with the most likely
      while p>cumrank[i]: #grab the random int when finally surpass the p value
        i+=1
      matingPool.append(pop[i]) #add it into the pool


    return matingPool


#function to mutate solutions
def mutate(x):

    #create some mutation logic  -- make sure to incorporate "mutationRate" somewhere and dont' do TOO much mutation
    p = myPRNG.random() #p is random

    if mutationRate > p: #only go here if you are lower than the small number of mutationRate
      i = myPRNG.randint(0,n-1)
      if(x[i]==0):#change one random bit
        x[i] = 1
      else:
        x[i] = 0

    return x

In [111]:
p

0.6507295825444175

In [112]:







#breeding -- uses the "mating pool" and calls "crossover" function
def breeding(matingPool):
    #the parents will be the first two individuals, then next two, then next two and so on

    children = []
    childrenFitness = []
    for i in range(0,populationSize-1,2):
        child1,child2=crossover(matingPool[i],matingPool[i+1])

        child1=mutate(child1)
        child2=mutate(child2)

        children.append(child1)
        children.append(child2)

        childrenFitness.append(evaluate(child1))
        childrenFitness.append(evaluate(child2))

    tempZip = zip(children, childrenFitness)
    popVals = sorted(tempZip, key=lambda tempZip: tempZip[1], reverse = True)

    #the return object is a sorted list of tuples:
    #the first element of the tuple is the chromosome; the second element is the fitness value
    #for example:  popVals[0] is represents the best individual in the population
    #popVals[0] for a 2D problem might be  ([-70.2, 426.1], 483.3)  -- chromosome is the list [-70.2, 426.1] and the fitness is 483.3

    return popVals


#insertion step
def insert(pop,kids):

    kids = kids[:]
    #this is not a good solution here... essentially this is replacing the previous generation with the offspring and not implementing any type of elitism
    #at the VERY LEAST evaluate the best solution from "pop" to make sure you are not losing a very good chromosome from last generation
    #maybe want to keep the top 5? 10? solutions from pop -- it's up to you.
    for i in range(eliteSolutions):
      kids.append(pop[i])


    return kids



#perform a simple summary on the population: returns the best chromosome fitness, the average population fitness, and the variance of the population fitness
def summaryFitness(pop):
    a=np.array(list(zip(*pop))[1])
    return np.max(a), np.mean(a), np.min(a), np.std(a)


#the best solution should always be the first element...
def bestSolutionInPopulation(pop):
    print ("Best solution: ", pop[0][0])
    print ("Items selected: ", itemsSelected(pop[0][0]))
    print ("Value: ", pop[0][1])
    print ("Weight: ", calcWeight(pop[0][0]))



def main():
    #GA main code
    Population = initializePopulation()


    #optional: you can output results to a file -- i've commented out all of the file out put for now
    #f = open('out.txt', 'w')  #---uncomment this line to create a file for saving output


    for j in range(Generations):

        mates=tournamentSelection(Population,10)  #<--need to replace this with roulette wheel selection, e.g.:  mates=rouletteWheel(Population)
        Offspring = breeding(mates)
        Population = insert(Population, Offspring)

        #end of GA main code

        maxVal, meanVal, minVal, stdVal=summaryFitness(Population)          #check out the population at each generation
        print("Iteration: ", j, summaryFitness(Population))                 #print to screen; turn this off for faster results

        #f.write(str(minVal) + " " + str(meanVal) + " " + str(varVal) + "\n")  #---uncomment this line to write to  file

    #f.close()   #---uncomment this line to close the file for saving output

    print (summaryFitness(Population))
    bestSolutionInPopulation(Population)


if __name__ == "__main__":
    main()

Iteration:  0 (-379843.9, -534988.8208888889, -674374.9000000001, 57825.2853594132)
Iteration:  1 (-324014.6, -439078.16355555557, -576316.2, 45302.736772438984)
Iteration:  2 (-287431.69999999995, -369232.4475555555, -468180.2, 33276.2868352359)
Iteration:  3 (-187671.69999999995, -321726.7573333333, -421503.4, 28486.029068142652)
Iteration:  4 (-156490.4999999999, -276683.56177777774, -370265.6, 33818.10916182639)
Iteration:  5 (-129195.60000000006, -216530.33288888884, -305820.5999999999, 35807.98927127462)
Iteration:  6 (-104746.20000000003, -164767.9493333333, -235698.7, 21046.61922799451)
Iteration:  7 (-89666.69999999997, -136529.7702222222, -181381.89999999997, 16922.870442786447)
Iteration:  8 (-63945.7, -111456.48, -169486.49999999994, 20784.034790181293)
Iteration:  9 (-37594.00000000006, -81841.148, -141120.70000000004, 17425.964231466867)
Iteration:  10 (-22264.800000000003, -60756.077333333335, -100426.50000000003, 14034.913198255012)
Iteration:  11 (8259.399999999967, -4